In [21]:
import pygsheets
import numpy as np
import pandas as pd

#authorization
gc = pygsheets.authorize(service_file='test-274615-040aa28420ed.json')

#open the google spreadsheet ('pysheeetsTest' exists)
sh = gc.open('pysheetsTest')

#select the tab A, B, we suppose that C doesn't exist
#we suppose that the key data (firstname, lastname and email) in tab A and B has been deduplicated.
A = sh.worksheet('title','A')
B = sh.worksheet('title','B')

In [22]:
# get the number of columns active within the sheet A and B
n_col_A = A.cols
n_col_B = B.cols

# get the number of rows active within the sheet A and B
n_row_A = A.rows
n_row_B = B.rows

# get the first row within the sheet A and B, which means the labels, turn it into an array form
row_A = np.array(A.get_row(1))
row_B = np.array(B.get_row(1))

In [23]:
# create the worksheet C
sh.add_worksheet("C")

<Worksheet 'C' index:2>

In [24]:
# turn A to a dataframe, use the first row as the labels
df_A = A.get_as_df()
n_label_A = df_A.shape[1]
n_row_A_ = df_A.shape[0]
df_A

,firstName,lastName,email,city,country,company,region
0,clément,galopin,sales@company.com,Paris,,Cartelis,Ile de France
1,zongmin,li,huonujian741@qq.com,Qingdao,China,None,None
2,wen,li,wen.li@qq.com,Beijing,,CAUC,None


In [25]:
# turn B to a dataframe, use the first row as the labels
df_B = B.get_as_df()
n_label_B = df_B.shape[1]
n_row_B_ = df_B.shape[0]
df_B

,firstName,lastName,email,region,city,country,sex
0,clément,galopin,sales@company.com,,London,FR,M
1,wen,li,,Dongli,Tianjin,China,M
2,zongmin,Li,huonujian741@qq.com,Dongli,Qingdao,China,M
3,zongmin,li,huonujian@qq.com,Dongli,None,None,None
4,zongmin,li,huonujian741@qq.com,Dongli,Palaiseau,France,M


In [26]:
# SQL requete
# select * from Table A full outer join Table B
# on Table A.firstName=Table B.firstName
# and Table A.lastName=Table B.lastName
# and Table A.email=Table B.email
df_C = df_A.merge(df_B, how='outer',left_on = ['firstName','lastName','email'], right_on = ['firstName','lastName','email'],suffixes=('_A','_B'))

In [27]:
df_C

,firstName,lastName,email,city_A,country_A,company,region_A,region_B,city_B,country_B,sex
0,clément,galopin,sales@company.com,Paris,,Cartelis,Ile de France,,London,FR,M
1,zongmin,li,huonujian741@qq.com,Qingdao,China,None,None,Dongli,Palaiseau,France,M
2,wen,li,wen.li@qq.com,Beijing,,CAUC,None,NaN,NaN,NaN,NaN
3,wen,li,,NaN,NaN,NaN,NaN,Dongli,Tianjin,China,M
4,zongmin,Li,huonujian741@qq.com,NaN,NaN,NaN,NaN,Dongli,Qingdao,China,M
5,zongmin,li,huonujian@qq.com,NaN,NaN,NaN,NaN,Dongli,None,None,None


In [28]:
# Replace all the None and '' by NaN, because we'll use the function combine_first 
# to realise the priority of tab A, and we need NaN, not None neither ''
df_C.fillna(value=pd.np.nan, inplace=True)
df_C.replace(to_replace=[''], value=pd.np.nan, inplace=True)

In [29]:
df_C

,firstName,lastName,email,city_A,country_A,company,region_A,region_B,city_B,country_B,sex
0,clément,galopin,sales@company.com,Paris,NaN,Cartelis,Ile de France,NaN,London,FR,M
1,zongmin,li,huonujian741@qq.com,Qingdao,China,NaN,NaN,Dongli,Palaiseau,France,M
2,wen,li,wen.li@qq.com,Beijing,NaN,CAUC,NaN,NaN,NaN,NaN,NaN
3,wen,li,NaN,NaN,NaN,NaN,NaN,Dongli,Tianjin,China,M
4,zongmin,Li,huonujian741@qq.com,NaN,NaN,NaN,NaN,Dongli,Qingdao,China,M
5,zongmin,li,huonujian@qq.com,NaN,NaN,NaN,NaN,Dongli,NaN,NaN,NaN


In [30]:
# find intersection between the set of A's labels and the set of B's labels (except 'firstName','lastName','email')

inter_label = [] # A∩B\{'firstName','lastName','email'}
for i in range(df_A.shape[1]):
    for j in range(df_B.shape[1]):
        if(df_A.axes[1][i] == df_B.axes[1][j]):
            inter_label.append(df_A.axes[1][i])
            break
inter_label.remove('firstName')
inter_label.remove('lastName')
inter_label.remove('email')

In [31]:
inter_label

['city', 'country', 'region']

In [32]:
# deduplicate the intersected columns, with A's priority greater than B's
tmp_col_1 = []
for i in range(len(inter_label)):
    tmp_A = inter_label[i]+'_A'
    tmp_B = inter_label[i]+'_B'
    tmp_col_1.append(df_C[tmp_A].combine_first(df_C[tmp_B]))
    df_C[tmp_A] = tmp_col_1[i]
    df_C = df_C.drop(columns = [tmp_B])

In [33]:
df_C

,firstName,lastName,email,city_A,country_A,company,region_A,sex
0,clément,galopin,sales@company.com,Paris,FR,Cartelis,Ile de France,M
1,zongmin,li,huonujian741@qq.com,Qingdao,China,NaN,Dongli,M
2,wen,li,wen.li@qq.com,Beijing,NaN,CAUC,NaN,NaN
3,wen,li,NaN,Tianjin,China,NaN,Dongli,M
4,zongmin,Li,huonujian741@qq.com,Qingdao,China,NaN,Dongli,M
5,zongmin,li,huonujian@qq.com,NaN,NaN,NaN,Dongli,NaN


In [34]:
# Now we just delete the suffixe of intersected labels
for i in range(len(inter_label)):
    for j in range(df_C.shape[1]):
        tmp_ixe = inter_label[i]+'_A'
        if(df_C.axes[1][j] == tmp_ixe):
            df_C = df_C.rename(columns={tmp_ixe:inter_label[i]})

In [35]:
df_C

,firstName,lastName,email,city,country,company,region,sex
0,clément,galopin,sales@company.com,Paris,FR,Cartelis,Ile de France,M
1,zongmin,li,huonujian741@qq.com,Qingdao,China,NaN,Dongli,M
2,wen,li,wen.li@qq.com,Beijing,NaN,CAUC,NaN,NaN
3,wen,li,NaN,Tianjin,China,NaN,Dongli,M
4,zongmin,Li,huonujian741@qq.com,Qingdao,China,NaN,Dongli,M
5,zongmin,li,huonujian@qq.com,NaN,NaN,NaN,Dongli,NaN


In [36]:
# Before the last step, replace all the NaN by '' because the client doesn't want to see NaN in his/her sheet
df_C = df_C.fillna('')

In [37]:
# The last step! Update df_C in the Google Sheet
C = sh.worksheet('title','C')
C.set_dataframe(df_C,(1,1))